In [18]:
# import the necessary packages
from imutils.perspective import four_point_transform
from imutils import contours
import numpy as np
import argparse
import imutils
import cv2
# construct the argument parse and parse the arguments
# ap = argparse.ArgumentParser()
# ap.add_argument("-i", "--image", required=True,
# 	help="path to the input image")
# args = vars(ap.parse_args())
# define the answer key which maps the question number
# to the correct answer
ANSWER_KEY = {0: 1, 1: 4, 2: 0, 3: 3, 4: 1, 5: 3, 6: 2, 7: 3}

In [19]:
# load the image, convert it to grayscale, blur it
# slightly, then find edges
image = cv2.imread("test2.png")
gray = cv2.cvtColor(image, cv2.COLOR_BGR2GRAY)
blurred = cv2.GaussianBlur(gray, (5, 5), 0)
edged = cv2.Canny(blurred, 75, 200)

In [20]:
cv2.imshow("a",edged)
cv2.waitKey(0)

-1

In [21]:
# find contours in the edge map, then initialize
# the contour that corresponds to the document
cnts = cv2.findContours(edged.copy(), cv2.RETR_EXTERNAL,
	cv2.CHAIN_APPROX_SIMPLE)
cnts = imutils.grab_contours(cnts)
docCnt = None
# ensure that at least one contour was found
if len(cnts) > 0:
	# sort the contours according to their size in
	# descending order
	cnts = sorted(cnts, key=cv2.contourArea, reverse=True)
	# loop over the sorted contours
	for c in cnts:
		# approximate the contour
		peri = cv2.arcLength(c, True)
		approx = cv2.approxPolyDP(c, 0.02 * peri, True)
		# if our approximated contour has four points,
		# then we can assume we have found the paper
		if len(approx) == 4:
			docCnt = approx
			break

In [22]:
print(docCnt)

[[[ 63  31]]

 [[ 62 514]]

 [[307 514]]

 [[306  32]]]


In [23]:
# apply a four point perspective transform to both the
# original image and grayscale image to obtain a top-down
# birds eye view of the paper
paper = four_point_transform(image, docCnt.reshape(4, 2))
warped = four_point_transform(gray, docCnt.reshape(4, 2))

In [24]:
cv2.imshow("a",warped)
cv2.waitKey(0)

-1

In [25]:
# apply Otsu's thresholding method to binarize the warped
# piece of paper
thresh1 = cv2.threshold(warped, 0, 255,
	cv2.THRESH_BINARY_INV | cv2.THRESH_OTSU)[1]

In [26]:
cv2.imshow("a",thresh1)
cv2.waitKey(0)

-1

In [27]:
# load the image, convert it to grayscale, blur it
# slightly, then find edges
image_mark = cv2.imread("test2 - Copy.png")
gray_mark = cv2.cvtColor(image_mark, cv2.COLOR_BGR2GRAY)
blurred_mark = cv2.GaussianBlur(gray_mark, (5, 5), 0)
edged_mark = cv2.Canny(blurred_mark, 75, 200)

In [28]:
cv2.imshow("a",edged_mark)
cv2.waitKey(0)

-1

In [29]:
# find contours in the edge map, then initialize
# the contour that corresponds to the document
cnts_mark = cv2.findContours(edged_mark.copy(), cv2.RETR_EXTERNAL,
	cv2.CHAIN_APPROX_SIMPLE)
cnts_mark = imutils.grab_contours(cnts_mark)
docCnt_mark = None
# ensure that at least one contour was found
if len(cnts_mark) > 0:
	# sort the contours according to their size in
	# descending order
	cnts_mark = sorted(cnts_mark, key=cv2.contourArea, reverse=True)
	# loop over the sorted contours
	for c in cnts_mark:
		# approximate the contour
		peri = cv2.arcLength(c, True)
		approx = cv2.approxPolyDP(c, 0.02 * peri, True)
		# if our approximated contour has four points,
		# then we can assume we have found the paper
		if len(approx) == 4:
			docCnt_mark = approx
			break

In [30]:
# apply a four point perspective transform to both the
# original image and grayscale image to obtain a top-down
# birds eye view of the paper
paper_mark = four_point_transform(image_mark, docCnt_mark.reshape(4, 2))
warped_mark = four_point_transform(gray_mark, docCnt_mark.reshape(4, 2))

In [31]:
# apply Otsu's thresholding method to binarize the warped
# piece of paper
thresh_mark = cv2.threshold(warped_mark, 0, 255,
	cv2.THRESH_BINARY_INV | cv2.THRESH_OTSU)[1]

In [32]:
cv2.imshow("a",thresh_mark)
cv2.waitKey(0)

-1

In [35]:
diff = cv2.absdiff(warped, warped_mark)
cv2.imshow("a",diff)
cv2.waitKey(0)

-1

In [40]:
thresh = cv2.threshold(diff, 0, 255,
cv2.THRESH_BINARY_INV | cv2.THRESH_OTSU)[1]
cv2.imshow("a",thresh)
cv2.waitKey(0)

-1

In [41]:
thresh = cv2.bitwise_not(thresh)
cv2.imshow("a",thresh)
cv2.waitKey(0)

-1

In [42]:
# find contours in the thresholded image, then initialize
# the list of contours that correspond to questions
cnts = cv2.findContours(thresh.copy(), cv2.RETR_EXTERNAL,
	cv2.CHAIN_APPROX_SIMPLE)
cnts = imutils.grab_contours(cnts)
questionCnts = []
# loop over the contours
for c in cnts:
	# compute the bounding box of the contour, then use the
	# bounding box to derive the aspect ratio
	(x, y, w, h) = cv2.boundingRect(c)
	ar = w / float(h)
	# in order to label the contour as a question, region
	# should be sufficiently wide, sufficiently tall, and
	# have an aspect ratio approximately equal to 1
	if w >= 20 and h >= 20 and ar >= 0.9 and ar <= 1.1:
		questionCnts.append(c)

In [43]:
# sort the question contours top-to-bottom, then initialize
# the total number of correct answers
questionCnts = contours.sort_contours(questionCnts,
	method="top-to-bottom")[0]
correct = 0
# each question has 5 possible answers, to loop over the
# question in batches of 5
for (q, i) in enumerate(np.arange(0, len(questionCnts), 6)):
	# sort the contours for the current question from
	# left to right, then initialize the index of the
	# bubbled answer
	cnts = contours.sort_contours(questionCnts[i:i + 6])[0]
	bubbled = None
	# loop over the sorted contours
	for (j, c) in enumerate(cnts):
		# construct a mask that reveals only the current
		# "bubble" for the question
		mask = np.zeros(thresh.shape, dtype="uint8")
		cv2.drawContours(mask, [c], -1, 255, -1)
		# apply the mask to the thresholded image, then
		# count the number of non-zero pixels in the
		# bubble area
		mask = cv2.bitwise_and(thresh, thresh, mask=mask)
		total = cv2.countNonZero(mask)
		# if the current total has a larger number of total
		# non-zero pixels, then we are examining the currently
		# bubbled-in answer
		if bubbled is None or total > bubbled[0]:
			bubbled = (total, j)
# initialize the contour color and the index of the
# *correct* answer
color = (0, 0, 255)
k = ANSWER_KEY[q]
# check to see if the bubbled answer is correct
if k == bubbled[1]:
	color = (0, 255, 0)
	correct += 1
# draw the outline of the correct answer on the test
cv2.drawContours(paper, [cnts[k]], -1, color, 3)

array([[[196, 196, 196],
        [171, 171, 171],
        [163, 163, 163],
        ...,
        [ 46,  46,  46],
        [ 81,  81,  81],
        [136, 136, 136]],

       [[112, 112, 112],
        [ 48,  48,  48],
        [ 35,  35,  35],
        ...,
        [  1,   1,   1],
        [ 13,  13,  13],
        [ 78,  78,  78]],

       [[ 58,  58,  58],
        [  0,   0,   0],
        [ 13,  13,  13],
        ...,
        [ 44,  44,  44],
        [ 16,  16,  16],
        [ 59,  59,  59]],

       ...,

       [[166, 166, 166],
        [ 60,  60,  60],
        [ 55,  55,  55],
        ...,
        [ 27,  27,  27],
        [ 67,  67,  67],
        [159, 159, 159]],

       [[169, 169, 169],
        [ 57,  57,  57],
        [ 19,  19,  19],
        ...,
        [ 15,  15,  15],
        [ 68,  68,  68],
        [163, 163, 163]],

       [[168, 168, 168],
        [ 59,  59,  59],
        [  0,   0,   0],
        ...,
        [ 15,  15,  15],
        [ 75,  75,  75],
        [169, 169, 169]]

In [ ]:
# grab the test taker
score = (correct / 7.0) * 100
print("[INFO] score: {:.2f}%".format(score))
cv2.putText(paper, "{:.2f}%".format(score), (10, 30),
	cv2.FONT_HERSHEY_SIMPLEX, 0.9, (0, 0, 255), 2)
cv2.imshow("Original", image)
cv2.imshow("Exam", paper)
cv2.waitKey(0)